# 🔥 FireRiskAI - Análisis de Riesgo de Incendios Forestales

## 📊 EDA Específico para Prevención de Incendios

Este notebook analiza el dataset Forest Cover Type desde la perspectiva de **prevención de incendios forestales**, mapeando tipos de cobertura vegetal a niveles de riesgo de incendio basados en características geográficas y climáticas.

### 🎯 Objetivos:
1. **Mapear inflamabilidad** por tipo de bosque
2. **Analizar correlación** geográfica con riesgo
3. **Identificar zonas críticas** de alto riesgo
4. **Crear visualizaciones** para toma de decisiones

In [ ]:
# ============================================
# 📚 IMPORTS Y CONFIGURACIÓN
# ============================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ucimlrepo import fetch_ucirepo
import warnings
warnings.filterwarnings('ignore')

# Configuración de estilo
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

print("- Análisis de Riesgo de Incendios Forestales")
print("=" * 60)


In [ ]:
# ============================================
# 📊 CARGA DE DATOS
# ============================================

print("📥 Cargando Forest Cover Type Dataset")
covertype = fetch_ucirepo(id=31)
X = covertype.data.features
y = covertype.data.targets.iloc[:, 0]

# Crear DataFrame combinado
df = pd.concat([X, y], axis=1)
df.columns = list(X.columns) + ['Cover_Type']

print(f"✅ Dataset cargado: {df.shape[0]:,} muestras, {df.shape[1]} features")
print(f"📋 Tipos de bosque: {sorted(df['Cover_Type'].unique())}")
print(f"\n📊 Primeras 5 filas:")
df.head()


In [ ]:
# ============================================
# 🔥 MAPEO DE INFLAMABILIDAD POR TIPO DE BOSQUE
# ============================================

print("🔥 MAPEO DE INFLAMABILIDAD POR TIPO DE BOSQUE")
print("=" * 50)

# Definir niveles de riesgo basados en características de inflamabilidad
fire_risk_mapping = {
    1: {
        "name": "Spruce/Fir (Abeto/Pícea)",
        "risk": "BAJO",
        "score": 2,
        "reason": "Coníferas densas pero húmedas, resistentes al fuego",
        "color": "green"
    },
    2: {
        "name": "Lodgepole Pine (Pino Lodgepole)",
        "risk": "ALTO",
        "score": 8,
        "reason": "Pinos densos, muy inflamables, propagación rápida",
        "color": "red"
    },
    3: {
        "name": "Ponderosa Pine (Pino Ponderosa)",
        "risk": "MEDIO",
        "score": 5,
        "reason": "Pinos dispersos, corteza gruesa, moderadamente resistente",
        "color": "orange"
    },
    4: {
        "name": "Cottonwood/Willow (Álamo/Sauce)",
        "risk": "BAJO",
        "score": 1,
        "reason": "Árboles de hoja ancha, cerca del agua, muy resistentes",
        "color": "green"
    },
    5: {
        "name": "Aspen (Álamo temblón)",
        "risk": "MEDIO",
        "score": 4,
        "reason": "Hojas caducas, pero troncos secos pueden arder",
        "color": "orange"
    },
    6: {
        "name": "Douglas-fir (Abeto de Douglas)",
        "risk": "MEDIO",
        "score": 6,
        "reason": "Coníferas grandes, corteza inflamable en condiciones secas",
        "color": "orange"
    },
    7: {
        "name": "Krummholz (Vegetación alpina)",
        "risk": "ALTO",
        "score": 9,
        "reason": "Vegetación alpina seca, vientos fuertes, propagación extrema",
        "color": "red"
    }
}

# Crear variables de riesgo
df['fire_risk_score'] = df['Cover_Type'].map({k: v['score'] for k, v in fire_risk_mapping.items()})
df['fire_risk_level'] = df['Cover_Type'].map({k: v['risk'] for k, v in fire_risk_mapping.items()})
df['forest_type_name'] = df['Cover_Type'].map({k: v['name'] for k, v in fire_risk_mapping.items()})

print("✅ Mapeo de inflamabilidad completado")
print("\n📋 Tabla de mapeo:")
for cover_type, info in fire_risk_mapping.items():
    print(f"{cover_type}. {info['name']} - {info['risk']} (Score: {info['score']})")
    print(f"   Razón: {info['reason']}")
    print()


In [ ]:
# ============================================
# 📊 DISTRIBUCIÓN DE RIESGO DE INCENDIO
# ============================================

print("📊 DISTRIBUCIÓN DE RIESGO DE INCENDIO")
print("=" * 40)

# Análisis de distribución
risk_distribution = df['fire_risk_level'].value_counts()
risk_percentage = df['fire_risk_level'].value_counts(normalize=True) * 100

print("\n📈 Distribución por nivel de riesgo:")
for level in ['BAJO', 'MEDIO', 'ALTO']:
    count = risk_distribution.get(level, 0)
    pct = risk_percentage.get(level, 0)
    print(f"{level}: {count:,} muestras ({pct:.1f}%)")

# Visualización de distribución
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Gráfico de barras por nivel de riesgo
risk_counts = df['fire_risk_level'].value_counts()
colors = ['green', 'orange', 'red']
bars = ax1.bar(risk_counts.index, risk_counts.values, color=colors, alpha=0.7)
ax1.set_title('Distribución de Muestras por Nivel de Riesgo de Incendio', fontsize=14, fontweight='bold')
ax1.set_xlabel('Nivel de Riesgo')
ax1.set_ylabel('Número de Muestras')
ax1.grid(axis='y', alpha=0.3)

# Añadir valores en las barras
for bar, count in zip(bars, risk_counts.values):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1000, 
             f'{count:,}', ha='center', va='bottom', fontweight='bold')

# Gráfico de pastel
ax2.pie(risk_counts.values, labels=risk_counts.index, colors=colors, autopct='%1.1f%%', startangle=90)
ax2.set_title('Proporción de Riesgo de Incendio', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

# Análisis por tipo de bosque
print("\n🌲 Distribución detallada por tipo de bosque:")
forest_risk = df.groupby(['forest_type_name', 'fire_risk_level']).size().unstack(fill_value=0)
print(forest_risk)


In [ ]:
# ============================================
# 🏔️ ANÁLISIS GEOGRÁFICO DE RIESGO
# ============================================

print("🏔️ ANÁLISIS GEOGRÁFICO DE RIESGO")
print("=" * 35)

# Variables geográficas clave
geo_vars = ['Elevation', 'Slope', 'Aspect', 
           'Horizontal_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways']

print("\n📊 Estadísticas por nivel de riesgo:")
for var in geo_vars:
    print(f"\n{var}:")
    stats = df.groupby('fire_risk_level')[var].agg(['mean', 'std', 'min', 'max']).round(2)
    print(stats)

# Visualizaciones geográficas
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.ravel()

for i, var in enumerate(geo_vars):
    if i < len(axes):
        # Boxplot por nivel de riesgo
        sns.boxplot(data=df, x='fire_risk_level', y=var, ax=axes[i], 
                   order=['BAJO', 'MEDIO', 'ALTO'], palette=['green', 'orange', 'red'])
        axes[i].set_title(f'{var} por Nivel de Riesgo', fontweight='bold')
        axes[i].grid(axis='y', alpha=0.3)

# Gráfico adicional: correlación con riesgo
if len(geo_vars) < len(axes):
    # Correlación entre variables geográficas y riesgo
    corr_data = df[geo_vars + ['fire_risk_score']].corr()['fire_risk_score'].drop('fire_risk_score')
    bars = axes[-1].bar(range(len(corr_data)), corr_data.values, 
                       color=['skyblue' if x > 0 else 'lightcoral' for x in corr_data.values])
    axes[-1].set_title('Correlación con Riesgo de Incendio', fontweight='bold')
    axes[-1].set_xticks(range(len(corr_data)))
    axes[-1].set_xticklabels(corr_data.index, rotation=45)
    axes[-1].set_ylabel('Correlación')
    axes[-1].grid(axis='y', alpha=0.3)
    
    # Añadir valores en las barras
    for bar, value in zip(bars, corr_data.values):
        axes[-1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
                     f'{value:.3f}', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

# Análisis de correlación detallado
print("\n🔍 Correlación con riesgo de incendio (ordenado por importancia):")
correlations = df[geo_vars + ['fire_risk_score']].corr()['fire_risk_score'].drop('fire_risk_score')
correlations_sorted = correlations.abs().sort_values(ascending=False)
for var, corr in correlations_sorted.items():
    direction = "📈" if correlations[var] > 0 else "📉"
    print(f"{direction} {var}: {correlations[var]:.3f}")
